In [36]:
%cd "F:\University\Master's Thesis\Datasets\rnd"

F:\University\Master's Thesis\Datasets\rnd


In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import svm, ensemble, neural_network, naive_bayes, neighbors
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [16]:
import copy
from pathlib import Path

# List of columns you want to read
selected_columns = ["Timestamp [s]", "CPU usage [%]", "Memory usage [%]", "Service Name", "HTTP Method", "HTTP Status"]

file_path = 'VM001.csv'
df = pd.read_csv(file_path, usecols=selected_columns)
df.head(-1)

,Timestamp [s],CPU usage [%],Memory usage [%],HTTP Method,HTTP Status,Service Name
0,2013-06-30 22:03:00,0.000000,6.500000,GET,200,ProductCatalogService
1,2013-06-30 22:04:00,0.066578,6.536489,GET,200,ProductCatalogService
2,2013-06-30 22:05:00,0.157880,6.586528,GET,200,ProductCatalogService
3,2013-06-30 22:06:00,0.220217,6.620693,GET,200,ProductCatalogService
4,2013-06-30 22:07:00,0.259892,6.642437,GET,200,ProductCatalogService
...,...,...,...,...,...,...
239030,2013-12-13 21:53:00,5.958158,11.280959,GET,200,ProductCatalogService
239031,2013-12-13 21:54:00,6.337063,11.567609,GET,200,ProductCatalogService
239032,2013-12-13 21:55:00,7.714891,12.674611,GET,200,ProductCatalogService
239033,2013-12-13 21:56:00,10.173214,14.600788,GET,200,ProductCatalogService


In [17]:
# Function to classify failures based on CPU and Memory values
def classify_failure(row):
    memory_usage = row['Memory usage [%]']
    cpu_usage = row['CPU usage [%]']

    if memory_usage < 20 and cpu_usage < 20:
        return "Idle or Low-Traffic"
    elif memory_usage < 20 and 20 <= cpu_usage <= 70:
        return "Normal Operation"
    elif memory_usage < 20 and cpu_usage > 70:
        return "Processing Intensive Operation"
    elif 20 <= memory_usage <= 70 and cpu_usage < 20:
        return "Normal Operation"
    elif 20 <= memory_usage <= 70 and 20 <= cpu_usage <= 70:
        return "Normal Operation"
    elif 20 <= memory_usage <= 70 and cpu_usage > 70:
        return "Processing Intensive Operation"
    elif memory_usage > 70 and cpu_usage < 20:
        return "Potential Memory Leak"
    elif memory_usage > 70 and 20 <= cpu_usage <= 70:
        return "Memory-Intensive Operation"
    elif memory_usage > 70 and cpu_usage > 70:
        return "Resource-Intensive Operation"
    else:
        return "Unknown State"

# Apply the function to create the 'Failure Classification' column
df['Failure Classification'] = df.apply(classify_failure, axis=1)

# Display the updated DataFrame
df.head()

,Timestamp [s],CPU usage [%],Memory usage [%],HTTP Method,HTTP Status,Service Name,Failure Classification
0,2013-06-30 22:03:00,0.000000,6.500000,GET,200,ProductCatalogService,Idle or Low-Traffic
1,2013-06-30 22:04:00,0.066578,6.536489,GET,200,ProductCatalogService,Idle or Low-Traffic
2,2013-06-30 22:05:00,0.157880,6.586528,GET,200,ProductCatalogService,Idle or Low-Traffic
3,2013-06-30 22:06:00,0.220217,6.620693,GET,200,ProductCatalogService,Idle or Low-Traffic
4,2013-06-30 22:07:00,0.259892,6.642437,GET,200,ProductCatalogService,Idle or Low-Traffic


In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn import svm, ensemble, neural_network, naive_bayes, neighbors

# Assuming df is your pandas DataFrame
# Split the data into features (X) and target label (y)
X = df[['Memory usage [%]', 'CPU usage [%]']]

# Encode the categorical target variable using OneHotEncoder
encoder = OneHotEncoder(sparse=False)
y_encoded = encoder.fit_transform(df[['Service Name']].values.reshape(-1, 1))

# Create a DataFrame with the one-hot encoded columns
y = pd.DataFrame(y_encoded, columns=encoder.get_feature_names_out(['Service Name']))

# Convert the one-hot encoded columns back to the original categorical values
y = y.idxmax(axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest
rf_classifier_service = ensemble.RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier_service.fit(X_train, y_train)
y_pred_rf_service = rf_classifier_service.predict(X_test)
accuracy_rf_service = accuracy_score(y_test, y_pred_rf_service)
print(f'Random Forest Service Accuracy: {accuracy_rf_service}')


# Artificial Neural Network (ANN)
mlp_classifier_service = neural_network.MLPClassifier(hidden_layer_sizes=(100,50), max_iter=500, random_state=42)
mlp_classifier_service.fit(X_train, y_train)
y_pred_mlp_service = mlp_classifier_service.predict(X_test)
accuracy_mlp_service = accuracy_score(y_test, y_pred_mlp_service)
print(f'ANN Service Accuracy: {accuracy_mlp_service}')

# Naive Bayes
nb_classifier_service = naive_bayes.GaussianNB()
nb_classifier_service.fit(X_train, y_train)
y_pred_nb_service = nb_classifier_service.predict(X_test)
accuracy_nb_service = accuracy_score(y_test, y_pred_nb_service)
print(f'Naive Bayes Service Accuracy: {accuracy_nb_service}')

# K-Nearest Neighbors (KNN)
knn_classifier_service = neighbors.KNeighborsClassifier(n_neighbors=5)
knn_classifier_service.fit(X_train, y_train)
y_pred_knn_service = knn_classifier_service.predict(X_test)
accuracy_knn_service = accuracy_score(y_test, y_pred_knn_service)
print(f'KNN Service Accuracy: {accuracy_knn_service}')

# Support Vector Machine (SVM)
svm_classifier_service = svm.SVC()
svm_classifier_service.fit(X_train, y_train)
y_pred_svm_service = svm_classifier_service.predict(X_test)
accuracy_svm_service = accuracy_score(y_test, y_pred_svm_service)
print(f'SVM Service Accuracy: {accuracy_svm_service}')


F:\Software\Anaconda\envs\deeplearning-env\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Random Forest Service Accuracy: 0.7558776773761714
ANN Service Accuracy: 0.7530538821954484
Naive Bayes Service Accuracy: 0.717411311914324
KNN Service Accuracy: 0.7560240963855421
SVM Service Accuracy: 0.7527819611780455


In [19]:
# Save the models to disk
joblib.dump(rf_classifier_service, 'rf_classifier_service_model.joblib')
joblib.dump(mlp_classifier_service, 'mlp_classifier_service_model.joblib')
joblib.dump(nb_classifier_service, 'nb_classifier_service_model.joblib')
joblib.dump(knn_classifier_service, 'knn_classifier_service_model.joblib')
joblib.dump(svm_classifier_service, 'svm_classifier_service_model.joblib')

print("Service Name Models saved to disk.")

Service Name Models saved to disk.


In [20]:
# Load the models from disk
loaded_rf_classifier = joblib.load('rf_classifier_service_model.joblib')
loaded_mlp_classifier = joblib.load('mlp_classifier_service_model.joblib')
loaded_nb_classifier = joblib.load('nb_classifier_service_model.joblib')
loaded_knn_classifier = joblib.load('knn_classifier_service_model.joblib')
loaded_svm_classifier = joblib.load('svm_classifier_service_model.joblib')

In [21]:
# Random Forrest
y_pred_rf_service = loaded_rf_classifier.predict(X_test)
accuracy_rf_service = accuracy_score(y_test, y_pred_rf_service)
print(f'Random Forest Service Accuracy: {accuracy_rf_service}')


# Artificial Neural Network (ANN)
y_pred_mlp_service = loaded_mlp_classifier.predict(X_test)
accuracy_mlp_service = accuracy_score(y_test, y_pred_mlp_service)
print(f'ANN Service Accuracy: {accuracy_mlp_service}')

# Naive Bayes
y_pred_nb_service = loaded_nb_classifier.predict(X_test)
accuracy_nb_service = accuracy_score(y_test, y_pred_nb_service)
print(f'Naive Bayes Service Accuracy: {accuracy_nb_service}')

# K-Nearest Neighbors (KNN)
y_pred_knn_service = loaded_knn_classifier.predict(X_test)
accuracy_knn_service = accuracy_score(y_test, y_pred_knn_service)
print(f'KNN Service Accuracy: {accuracy_knn_service}')

# Support Vector Machine (SVM)
y_pred_svm_service = loaded_svm_classifier.predict(X_test)
accuracy_svm_service = accuracy_score(y_test, y_pred_svm_service)
print(f'SVM Service Accuracy: {accuracy_svm_service}')

Random Forest Service Accuracy: 0.7558776773761714
ANN Service Accuracy: 0.7530538821954484
Naive Bayes Service Accuracy: 0.717411311914324
KNN Service Accuracy: 0.7560240963855421
SVM Service Accuracy: 0.7527819611780455


In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn import svm, ensemble, neural_network, naive_bayes, neighbors

# Assuming df is your pandas DataFrame
# Split the data into features (X) and target label (y)
X = df[['Memory usage [%]', 'CPU usage [%]']]

# Encode the categorical target variable using OneHotEncoder
encoder_method = OneHotEncoder(sparse=False)
y_encoded_method = encoder_method.fit_transform(df[['HTTP Method']].values.reshape(-1, 1))

# Create a DataFrame with the one-hot encoded columns
y_method = pd.DataFrame(y_encoded_method, columns=encoder_method.get_feature_names_out(['HTTP Method']))

# Convert the one-hot encoded columns back to the original categorical values
y_method = y_method.idxmax(axis=1)

# Split the data into training and testing sets
X_train_method, X_test_method, y_train_method, y_test_method = train_test_split(X, y_method, test_size=0.2, random_state=42)

# Random Forest
rf_classifier_method = ensemble.RandomForestClassifier(n_estimators=300, random_state=42)
rf_classifier_method.fit(X_train_method, y_train_method)
y_pred_rf_method = rf_classifier_method.predict(X_test_method)
accuracy_rf_method = accuracy_score(y_test_method, y_pred_rf_method)
print(f'Random Forest HTTP Method Accuracy: {accuracy_rf_method}')


# Artificial Neural Network (ANN)
mlp_classifier_method = neural_network.MLPClassifier(hidden_layer_sizes=(100,50), max_iter=500, random_state=42)
mlp_classifier_method.fit(X_train_method, y_train_method)
y_pred_mlp_method = mlp_classifier_method.predict(X_test_method)
accuracy_mlp_method = accuracy_score(y_test_method, y_pred_mlp_method)
print(f'ANN HTTP Method Accuracy: {accuracy_mlp_method}')

# Naive Bayes
nb_classifier_method = naive_bayes.GaussianNB()
nb_classifier_method.fit(X_train_method, y_train_method)
y_pred_nb_method = nb_classifier_method.predict(X_test_method)
accuracy_nb_method = accuracy_score(y_test_method, y_pred_nb_method)
print(f'Naive Bayes HTTP Method Accuracy: {accuracy_nb_method}')

# K-Nearest Neighbors (KNN)
knn_classifier_method = neighbors.KNeighborsClassifier(n_neighbors=5)
knn_classifier_method.fit(X_train_method, y_train_method)
y_pred_knn_method = knn_classifier_method.predict(X_test_method)
accuracy_knn_method = accuracy_score(y_test_method, y_pred_knn_method)
print(f'KNN HTTP Method Accuracy: {accuracy_knn_method}')

# Support Vector Machine (SVM)
svm_classifier_method = svm.SVC()
svm_classifier_method.fit(X_train_method, y_train_method)
y_pred_svm_method = svm_classifier_method.predict(X_test_method)
accuracy_svm_method = accuracy_score(y_test_method, y_pred_svm_method)
print(f'SVM HTTP Method Accuracy: {accuracy_svm_method}')


F:\Software\Anaconda\envs\deeplearning-env\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Random Forest HTTP Method Accuracy: 0.7916248326639893
ANN HTTP Method Accuracy: 0.7849313922356091
Naive Bayes HTTP Method Accuracy: 0.7471134538152611
KNN HTTP Method Accuracy: 0.7901606425702812
SVM HTTP Method Accuracy: 0.7900978915662651


In [23]:
# Save the models to disk
joblib.dump(rf_classifier_method, 'rf_classifier_method_model.joblib')
joblib.dump(mlp_classifier_method, 'mlp_classifier_method_model.joblib')
joblib.dump(nb_classifier_method, 'nb_classifier_method_model.joblib')
joblib.dump(knn_classifier_method, 'knn_classifier_method_model.joblib')
joblib.dump(svm_classifier_method, 'svm_classifier_method_model.joblib')

print("Method Models saved to disk.")

Method Models saved to disk.


In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn import svm, ensemble, neural_network, naive_bayes, neighbors

# Assuming df is your pandas DataFrame
# Split the data into features (X) and target label (y)
X = df[['Memory usage [%]', 'CPU usage [%]']]

# Encode the categorical target variable using OneHotEncoder
encoder_status = OneHotEncoder(sparse=False)
y_encoded_status = encoder_status.fit_transform(df[['HTTP Status']].values.reshape(-1, 1))

# Create a DataFrame with the one-hot encoded columns
y_status = pd.DataFrame(y_encoded_status, columns=encoder_status.get_feature_names_out(['HTTP Status']))

# Convert the one-hot encoded columns back to the original categorical values
y_status = y_status.idxmax(axis=1)

# Split the data into training and testing sets
X_train_status, X_test_status, y_train_status, y_test_status = train_test_split(X, y_status, test_size=0.2, random_state=42)

# Random Forest
rf_classifier_status = ensemble.RandomForestClassifier(n_estimators=300, random_state=42)
rf_classifier_status.fit(X_train_status, y_train_status)
y_pred_rf_status = rf_classifier_status.predict(X_test_status)
accuracy_rf_status = accuracy_score(y_test_status, y_pred_rf_status)
print(f'Random Forest HTTP Status Accuracy: {accuracy_rf_status}')


# Artificial Neural Network (ANN)
mlp_classifier_status = neural_network.MLPClassifier(hidden_layer_sizes=(100,50), max_iter=500, random_state=42)
mlp_classifier_status.fit(X_train_status, y_train_status)
y_pred_mlp_status = mlp_classifier_status.predict(X_test_status)
accuracy_mlp_status = accuracy_score(y_test_status, y_pred_mlp_status)
print(f'ANN HTTP Status Accuracy: {accuracy_mlp_status}')

# Naive Bayes
nb_classifier_status = naive_bayes.GaussianNB()
nb_classifier_status.fit(X_train_status, y_train_status)
y_pred_nb_status = nb_classifier_status.predict(X_test_status)
accuracy_nb_status = accuracy_score(y_test_status, y_pred_nb_status)
print(f'Naive Bayes HTTP Status Accuracy: {accuracy_nb_status}')

# K-Nearest Neighbors (KNN)
knn_classifier_status = neighbors.KNeighborsClassifier(n_neighbors=5)
knn_classifier_status.fit(X_train_status, y_train_status)
y_pred_knn_status = knn_classifier_status.predict(X_test_status)
accuracy_knn_status = accuracy_score(y_test_status, y_pred_knn_status)
print(f'KNN HTTP Status Accuracy: {accuracy_knn_status}')

# Support Vector Machine (SVM)
svm_classifier_status = svm.SVC()
svm_classifier_status.fit(X_train_status, y_train_status)
y_pred_svm_status = svm_classifier_status.predict(X_test_status)
accuracy_svm_status = accuracy_score(y_test_status, y_pred_svm_status)
print(f'SVM HTTP Status Accuracy: {accuracy_svm_status}')

F:\Software\Anaconda\envs\deeplearning-env\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Random Forest HTTP Status Accuracy: 0.9752970214190094
ANN HTTP Status Accuracy: 0.967745983935743
Naive Bayes HTTP Status Accuracy: 0.9528949129852744
KNN HTTP Status Accuracy: 0.9769703815261044
SVM HTTP Status Accuracy: 0.9761546184738956


In [25]:
import joblib

# Save the models to disk
joblib.dump(rf_classifier_status, 'rf_classifier_status_model.joblib')
joblib.dump(mlp_classifier_status, 'mlp_classifier_status_model.joblib')
joblib.dump(nb_classifier_status, 'nb_classifier_status_model.joblib')
joblib.dump(knn_classifier_status, 'knn_classifier_status_model.joblib')
joblib.dump(svm_classifier_status, 'svm_classifier_status_model.joblib')

print("Status Models saved to disk.")

Status Models saved to disk.


In [31]:
# Testing on a model
# Given data
data = {
    'Memory usage [%]': [45.777958, 71.982993, 72.224752, 72.135143, 72.427164, 34.359488, 72.581069, 12.346585, 72.267944, 23.095423, 72.295631, 18.39634, 23.972726],
    'CPU usage [%]': [4.007266, 14.0537, 34.058797, 44.361495, 53.926062, 84.143917, 14.07761, 94.304139, 14.215333, 4.066917, 34.196585, 24.232192, 13.875389]
}

# Create DataFrame
df_testing = pd.DataFrame(data)

In [34]:
def classify_failure(row):
    memory_usage = row['Memory usage [%]']
    cpu_usage = row['CPU usage [%]']

    if memory_usage < 20 and cpu_usage < 20:
        return "Idle or Low-Traffic"
    elif memory_usage < 20 and 20 <= cpu_usage <= 70:
        return "Normal Operation"
    elif memory_usage < 20 and cpu_usage > 70:
        return "Processing Intensive Operation"
    elif 20 <= memory_usage <= 70 and cpu_usage < 20:
        return "Normal Operation"
    elif 20 <= memory_usage <= 70 and 20 <= cpu_usage <= 70:
        return "Normal Operation"
    elif 20 <= memory_usage <= 70 and cpu_usage > 70:
        return "Processing Intensive Operation"
    elif memory_usage > 70 and cpu_usage < 20:
        return "Potential Memory Leak"
    elif memory_usage > 70 and 20 <= cpu_usage <= 70:
        return "Memory-Intensive Operation"
    elif memory_usage > 70 and cpu_usage > 70:
        return "Resource-Intensive Operation"
    else:
        return "Unknown State"

# Apply the function to create the 'Failure Classification' column
df_testing['Failure Classification'] = df_testing.apply(classify_failure, axis=1)

# Display the updated DataFrame
df_testing.head(20)

,Memory usage [%],CPU usage [%],Failure Classification
0,45.777958,4.007266,Normal Operation
1,71.982993,14.053700,Potential Memory Leak
2,72.224752,34.058797,Memory-Intensive Operation
3,72.135143,44.361495,Memory-Intensive Operation
4,72.427164,53.926062,Memory-Intensive Operation
5,34.359488,84.143917,Processing Intensive Operation
6,72.581069,14.077610,Potential Memory Leak
7,12.346585,94.304139,Processing Intensive Operation
8,72.267944,14.215333,Potential Memory Leak
9,23.095423,4.066917,Normal Operation


In [37]:
loaded_rf_method_classifier = joblib.load('rf_classifier_method_model.joblib')
loaded_knn_status_classifier = joblib.load('knn_classifier_status_model.joblib')
loaded_knn_service_classifier = joblib.load('knn_classifier_service_model.joblib')

In [46]:
y_pred_method = loaded_rf_method_classifier.predict(df_testing[["Memory usage [%]", "CPU usage [%]"]])
y_pred_service = loaded_knn_service_classifier.predict(df_testing[["Memory usage [%]", "CPU usage [%]"]])
y_pred_status = loaded_knn_status_classifier.predict(df_testing[["Memory usage [%]", "CPU usage [%]"]])

In [54]:
for i in y_pred_status:
    print(i[12:])

200
429
429
429
429
503
429
503
429
200
429
200
200
